<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone Project - TheiaVision: Object Detection Technology for PMD Safety Alerts

> Authors: Ng Wei
---

**Problem Statement:**  
How can we enhance the safety of Personal Mobility Devices (PMDs) in urban environments by using object detection to improve PMD users' ability to perceive and respond to their surroundings?

**Target Audience:**
Management Team of PMD Maker

**Summary:**
This project aims to develop a object detection system to identifies obstacles such as pedestrians, vehicles, and traffic signs. By leveraging CNN algorithms YOLO model, this system would help Darren, a project manager, to lead the development of an alert system with object detection technology

There are a total of three notebooks for this project:  
 1. `01_EDA.ipynb`   
 2. `02_Modelling_Pytorch_SimpleCNN_SGTrafficSign.ipynb`   
 3. `03_Modelling_YOLOv8_labeled_SGTrafficSign.ipynb`
 4. `04_Merge_MultiDataset.ipynb`
 5. `05_Modelling_YOLOv8_combined_data.ipynb`
 6. `06_YOLOv8_Hyperparameter_Tuning.ipynb`

---
**This Notebook**
- We will train the YOLOv8 model with merged dataset `Master_dataset`

# 5.1 YOLOv8 Modelling with `Master_Dataset`

The dataset has been successfully prepared and split into training, validation, and testing sets. This includes ensuring a balanced distribution of classes across all sets, especially with a focus on rare class representation.

In [23]:
import os

# Define the classes and their corresponding indices
class_names = [
    'curveleft_sign',           # Class 0
    'curveright_sign',          # Class 1
    'mandatorystop_sign',       # Class 2
    'noentry_sign',             # Class 3
    'nojaywalking_sign',        # Class 4
    'pedestriancrossing_sign',  # Class 5
    'zebracrossing_sign',       # Class 6
    'biker',                    # Class 7
    'car',                      # Class 8
    'pedestrian',               # Class 9
    'trafficlight',             # Class 10
    'trafficlight_green',       # Class 11
    'trafficlight_red',         # Class 12
    'trafficlight_yellow',      # Class 13
    'truck'                     # Class 14
]
class_indices = {name: index for index, name in enumerate(class_names)}

# Image dimensions
img_width, img_height = 640, 640

# Path to the folder containing the annotation files
folder_path = 'Master_Dataset/train/labels'
output_folder_path = 'Master_Dataset/train/modified_labels'  # New output folder

os.makedirs(output_folder_path, exist_ok=True)  # Ensure the output directory exists

# Function to normalize coordinates
def normalize_coordinates(x, y, w, h, img_width, img_height):
    dw = 1.0 / img_width
    dh = 1.0 / img_height
    x_center = (x + w / 2.0) * dw
    y_center = (y + h / 2.0) * dh
    width = w * dw
    height = h * dh
    return x_center, y_center, width, height

# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # assuming the annotation files are .txt files
        file_path = os.path.join(folder_path, filename)
        output_file_path = os.path.join(output_folder_path, filename)  # Output file path
        with open(file_path, 'r') as file:
            lines = file.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:  # Ensure this is a YOLO format line
                class_index, x, y, w, h = map(float, parts)
                # Normalize coordinates
                x_center, y_center, width, height = normalize_coordinates(x, y, w, h, img_width, img_height)
                new_line = f"{int(class_index)} {x_center} {y_center} {width} {height}"
                new_lines.append(new_line)

        # Write the converted lines to a new file in the output directory
        with open(output_file_path, 'w') as file:
            file.write('\n'.join(new_lines))


## 5.1.1 Model Training Execution Using YOLOv8

### 5.1.1.1 Overview
This script automates the process of training a YOLOv8 model using the `subprocess` module in Python. The training configuration specifies the model, dataset, number of epochs, image size, and batch size.

### 5.1.1.2 Script Details
- **Model:** `yolov8s.pt`
- **Dataset:** Specified in `data.yaml` located at `/home/mangguai/capstone/code/Master_Dataset/`
- **Epochs:** 20
- **Image Size:** 640x640 pixels
- **Batch Size:** 8

### 5.1.1.3 Command Execution
The training command is executed in the system's shell using `subprocess.run`. This method allows the script to initiate the YOLO model training directly from the Python environment.

In [2]:
import subprocess

# Command to run
command = "yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Master_Dataset/data.yaml epochs=20 imgsz=640 batch=8"

# Execute the command
subprocess.run(command, shell=True)

#134min for 20 epochs

New https://pypi.org/project/ultralytics/8.2.10 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/mangguai/capstone/code/Master_Dataset/data.yaml, epochs=20, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_wid

CompletedProcess(args='yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Master_Dataset/data.yaml epochs=20 imgsz=640 batch=8', returncode=0)

The YOLOv8 model completed 20 epochs of training on the dataset with 39,476 instances across 5,430 images, achieving a final mAP50 of 0.862 and mAP50-95 of 0.66, demonstrating strong performance especially in identifying traffic-related signs and objects. The entire training process took approximately 2.214 hours, effectively utilizing the capabilities of an NVIDIA GeForce RTX 3060 Laptop GPU.

---

# Code for testing

In [24]:
import subprocess

# Command to run
command = "yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Master_Dataset/data.yaml epochs=25 imgsz=640 batch=8"

# Execute the command
subprocess.run(command, shell=True)

#165min for 25 epochs

New https://pypi.org/project/ultralytics/8.2.10 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/mangguai/capstone/code/Master_Dataset/data.yaml, epochs=25, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_wid

CompletedProcess(args='yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Master_Dataset/data.yaml epochs=25 imgsz=640 batch=8', returncode=0)

In [40]:
#before running this command, make sure you are in the correct directory, check with pwd
!yolo task=detect mode=predict model=/home/mangguai/capstone/code/runs/detect/train5/weights/best.pt conf=0.25 source=/home/mangguai/capstone/data/greenlight.jpg

Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 11131389 parameters, 0 gradients, 28.5 GFLOPs

image 1/1 /home/mangguai/capstone/data/greenlight.jpg: 416x640 1 biker, 2 pedestrians, 92.6ms
Speed: 3.1ms preprocess, 92.6ms inference, 99.3ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict9
💡 Learn more at https://docs.ultralytics.com/modes/predict
